### Modules

In [ ]:
weber.junior@img.com.br
11 96378-9218

In [509]:
cd ../

C:\Users\luisr\Desktop\Repositories\Data Science Projects\Hackaton COR IV - Centro de Operações do RJ


In [415]:
import requests, json, pandas as pd, numpy as np
from matplotlib.path import Path as mpl_path
from IPython.display import clear_output as co

from Modulos.imbalanced_selection import groupConsecutiveFlags

---
# Get prediction alerts from waterbag models api

### Request predictions for region 'Catete'

In [128]:
waterbag_api = 'http://127.0.0.1:5000/'
catete_url = waterbag_api + 'predictions?cluster_id=1'

# api request
catete = pd.DataFrame(requests.get(catete_url).json())

catete.head()

,_id,timestamp,date,time,cluster_id,cluster,range,probability,confidence,label
0,6392b78ddae0e50eacc93ae6,2022-12-09 04:20:00.022000,2022-12-09,01:20:00,1,Rua do Catete,1h,0.000449,0.999103,0
1,6392a5eedae0e50eacc93a92,2022-12-09 03:05:00.022000,2022-12-09,00:05:00,1,Rua do Catete,1h,0.000488,0.999024,0
2,63929d0cdae0e50eacc93a62,2022-12-09 02:25:00.061000,2022-12-08,23:25:00,1,Rua do Catete,1h,0.000920,0.998159,0
3,63929201dae0e50eacc93a2c,2022-12-09 01:40:00.018000,2022-12-08,22:40:00,1,Rua do Catete,1h,0.000800,0.998399,0
4,639290dfdae0e50eacc93a26,2022-12-09 01:35:00.024000,2022-12-08,22:35:00,1,Rua do Catete,1h,0.000849,0.998302,0


### Type conversion

In [132]:
catete['timestamp'] = catete['timestamp'].astype('datetime64[ms]')

catete.set_index('timestamp', inplace=True)
catete.sort_index(inplace=True)

### Operation start

In [133]:
operation_start = catete.index.min()

str(operation_start.date())

'2022-10-19'

### Operation time

In [134]:
pd.datetime.now() - operation_start

<ipython-input-134-18cfd65ff83b>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime.now() - operation_start


Timedelta('53 days 02:11:38.060275')

In [135]:
yprob = catete['probability']
yhat = (yprob >= 0.5).astype('int')
ygroup = groupConsecutiveFlags(yhat)

catete['alert'] = ygroup

### Predicted alerts duration

In [144]:
alert_duration = 5 * ygroup.value_counts().drop(-1).to_frame('duration (min)')

alert_duration.T

,6,0,7,2,8,9,1,3,4,5
duration (min),165,95,80,70,70,70,60,35,10,10


### Alerts date and time

In [147]:
start = catete.groupby('alert').first()[['date', 'time']]
end = catete.groupby('alert').last()[['date', 'time']]
end.columns = ['date_end', 'time_end']

alerts = start.join([end[['time_end']], alert_duration]).drop(-1)

alerts

,date,time,time_end,duration (min)
alert,,,,
0,2022-10-31,17:10:00,18:40:00,95.0
1,2022-11-01,13:55:00,14:50:00,60.0
2,2022-11-01,15:20:00,16:25:00,70.0
3,2022-11-01,17:55:00,18:25:00,35.0
4,2022-11-01,18:55:00,19:00:00,10.0
5,2022-11-24,05:45:00,05:50:00,10.0
6,2022-11-24,06:00:00,08:40:00,165.0
7,2022-11-28,13:45:00,15:00:00,80.0
8,2022-11-28,15:15:00,16:20:00,70.0


---
# Get *closed* waterbag events from Sistema Comando

### Set api urls and request parameters 

In [424]:
comando_api = 'https://api.dados.rio/v2/adm_cor_comando/'
operation_events_url = comando_api + 'ocorrencias'

dict_df_map = lambda df: pd.DataFrame(df['eventos'])

days_30 = pd.offsets.Day() * 30
now = pd.datetime.now()

n_requests = int(np.ceil((now - operation_start) / (days_30)))

<ipython-input-424-1300566dc13c>:7: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  now = pd.datetime.now()


### Request events from comando api
#### From operation start time until now, in periods of 30 days

In [473]:
start = operation_start

events = []
events_res = {}
for n in range(n_requests):
    co(True); print(n+1, '/', n_requests, 'periods requested')
    if n != 0: start = start + days_30
    start_str = str(start)[:-6] + '0'
    end = start + days_30
    end_str = str(end)[:-6] + '0'
    rqst_url = operation_events_url + f'?inicio={start_str}&fim={end_str}'
    j = 0
    while 'eventos' not in events_res.keys():
        j+=1; events_res = requests.get(rqst_url).json()
        if j>=10:
            print('retries exceeded'); break
    events_res = pd.concat(list(map(dict_df_map, events_res['eventos'])))
    events.append(events_res)

events = pd.concat(events).reset_index(drop=True)

2 / 2 periods requested


### Data cleaning

In [474]:
events['inicio'] = events['inicio'].astype('datetime64[ms]')
events['fim'] = events['fim'].astype('datetime64[ms]')

### Events temporal description

In [475]:
events['inicio'].describe(datetime_is_numeric=True)[['min', 'max']]

min    2022-10-19 05:59:54
max    2022-12-12 12:48:43
Name: inicio, dtype: object

### Get events inside the model's region of influence

#### Get the region polygon

In [476]:
polygons_geojson_path = '../ACELERAÇÃO/Dados/Clusters/polygons_micro.geojson'

# Load custers polygons geojson data
polygons_geojson = json.loads(open(polygons_geojson_path, 'r').read())

# Get polygons dict from geojson
polygons = {}
for poly in polygons_geojson['features']:
    if poly['properties']['sublabel'] != -1:
        polygons[poly['properties']['sublabel']] = poly['geometry']['coordinates'][0]

#### Get events inside polygon

In [477]:
#### Get events points as array
points = np.array(list(map(tuple, events[['longitude', 'latitude']].values)))

cluster_id = 1
poly = polygons[cluster_id]

# Get events inside polygon
points_msk = mpl_path(poly).contains_points(points)
events_poly = events[points_msk]

events_poly

,tipo,pop_id,bairro,latitude,inicio,titulo,fim,prazo,descricao,informe_id,gravidade,id,longitude,status
742,PRIMARIO,5.0,Catete,-22.927022,2022-11-01 15:39:00,Bolsão d'água em via ( CAM 313 ),2022-11-01 17:58:04,CURTO,"R. do Catete, Alt. R. Silveira Martins - Catete",83611,BAIXO,83610,-43.176622,FECHADO
2333,PRIMARIO,31.0,Glória,-22.925695,2022-11-28 13:33:46,Alagamento (Cam 313),2022-11-28 16:17:42,CURTO,"Rua do Catete, Alt. R. Silveira Martins - Catete",85187,MEDIO,85186,-43.176198,FECHADO
2336,PRIMARIO,5.0,Glória,-22.926966,2022-11-28 13:40:46,Bolsão d'água em via (Cam 268),2022-11-28 15:15:56,CURTO,R. do Catete - Largo do Machado,85190,BAIXO,85189,-43.176611,FECHADO


### Get events by date

In [ ]:
'2022-10-31', '2022-11-01', '2022-11-24', '2022-11-28'

events.set_index('inicio')['2022-11-24']#.set_index('pop_id')[5]